<a href="https://colab.research.google.com/github/goromal/FANet_Evaluation/blob/main/fanet_eval_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FANet Evaluation - 6.862 Project



*   David Elatov
*   Dayne Howard
*   Andrew Torgesen



## Setup

### CUDA

1. Go to **Menu > Runtime > Change runtime type** and make sure that GPU is enabled.
2. Run the commands below to ensure that the GPU (and CUDA) is operational.

In [1]:
! nvidia-smi

Tue Apr  6 14:06:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
torch.cuda.is_available()

True

### Repository

Mount drive, clone repo, navigate to repo, and change working directory to access repo files. **Run ONCE per computing session.**

In [1]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/
if not os.path.exists('FANet_Evaluation'):
  print('Repo not present. Cloning...')
  ! git clone https://github.com/goromal/FANet_Evaluation.git
  %cd FANet_Evaluation/
else:
  print('Repo already present. Updating...')
  %cd FANet_Evaluation/
  ! git pull origin main
from model.test_model import *
test()

Mounted at /content/gdrive
/content/gdrive/MyDrive
Repo already present. Updating...
/content/gdrive/MyDrive/FANet_Evaluation
From https://github.com/goromal/FANet_Evaluation
 * branch            main       -> FETCH_HEAD
Already up to date.
SUCCESS


### Python Packages

In [ ]:
! pip install oyaml
! pip install torchstat

## FANet-18 Initial FPS esting

In [ ]:
import torch
import oyaml as yaml
from torchstat import stat
import time,os

from model.fanet import FANet

In [ ]:
network = FANet(backbone='resnet18')
network.cuda()
network.eval()
t_cnt = 0.0
with torch.no_grad():
  input = torch.rand((1,3,1024,2048)).cuda()
  
  torch.cuda.synchronize()
  x = network(input)
  x = network(input)
  
  torch.cuda.synchronize()
  torch.cuda.synchronize()
  start_ts = time.time()

  for i in range(100):
    x = network(input)
  
  torch.cuda.synchronize()
  end_ts = time.time()

  t_cnt = end_ts-start_ts

print('FANet-18 Performance (FPS): %f' % (100.0/t_cnt))

FANet-18 Performance (FPS): 40.201780


# Evaluation Pipeline Baseline

In [ ]:
# TODO decompose agent.py, majority of functionality in train.py

In [3]:
import sys
sys.path.insert(0, '/content/gdrive/MyDrive/FANet_Evaluation/evaluation') # so that the evaluation pipeline's internal imports work
from evaluation.utils.params import get_params
from evaluation.utils.dirs import create_exp_dirs

In [4]:
# Usable Models
from evaluation.models.dilation_mobilenet import DilationMobileNet

AttributeError: ignored